In [1]:
# Import packages
import pyspark
import dxpy
import dxdata
from pyspark.sql import SQLContext

In [2]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
# Need to adjust this buffer otherwise will get an error in toPandas() call
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "128m")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

/cluster/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [4]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [5]:
def get_proteins():
    filename = "/mnt/project/fields/field_names.txt"
    with open(filename, "r") as f:
        proteins = [p.strip() for p in f.readlines() if not p=="eid\n"]
    return proteins

In [6]:
proteins = get_proteins()

In [7]:
field_name_dict = {
    'sample_names': 'eid'
}

for p in proteins:
    field_name_dict[p.upper()] = p

In [8]:
def retrieve_olink_instance(dataset, inst, field_name_dict):
    olink_i = dataset[f'olink_instance_{inst}']
    field_names = list(field_name_dict.values())
    df = olink_i.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="replace")
    df = df.toPandas()
    df = df.rename(columns={v:k for k,v in field_name_dict.items()})
    return df

In [9]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return

In [10]:
proj_dir = "/fields/data/proteomics/"
for inst in [2, 3]:
    print(inst)
    df = retrieve_olink_instance(dataset, inst, field_name_dict)
    filename = f"npx_inst{inst}.csv.gz"
    df.to_csv(filename, index=False)
    upload_file_to_project(filename, proj_dir)


2


/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

*********npx_inst2.csv.gz uploaded!!*********
3


/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/cluster/spark/python/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

*********npx_inst3.csv.gz uploaded!!*********


# Resources

1. https://dnanexus.gitbook.io/uk-biobank-rap/working-on-the-research-analysis-platform/using-spark-to-analyze-tabular-data
2. https://github.com/dnanexus/OpenBio/blob/master/UKB_notebooks/ukb-rap-pheno-basic.ipynb
3. https://community.dnanexus.com/s/question/0D582000001ewWWCAY/could-you-please-offer-the-list-of-field-names-for-all-proteins-so-that-we-can-download-all-the-proteomic-data-